In [27]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import cv2
from prettytable import PrettyTable
from sklearn.model_selection import train_test_split
import random
from skimage.util import view_as_windows
from matplotlib.pyplot import imread, imsave

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping, TensorBoard,ReduceLROnPlateau
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPooling2D,MaxPooling1D, Concatenate, Flatten, Dropout, Dense, Conv1D,MaxPool2D,BatchNormalization
import tensorflow as tf
import datetime

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from scipy.sparse import csr_matrix

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
import keras
from keras.applications import *

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from keras.models import Model, load_model
import pickle


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
#Initialise all the director paths and fetch the filenames from this directory. 
#The dataset consists of images belonging to Van Gogh and other artists. The images are segregated into separate folders for train and test.
#The train directory and test directory each have two folders - vg and nvg. 
#vg folder has images belonging to Van Gogh
#nvg folder has images belonginh to Non Van Gogh


rootdir = '/content/drive/MyDrive/Colab Notebooks/29. Identification of Van Gogh paintings/vgdb_2016/vgdb_2016'

#Train data
traindir = rootdir + '/train'
traindir_vg = traindir + '/vg'
traindir_nvg = traindir + '/nvg'

#Test data
testdir = rootdir + '/test'
testdir_vg = testdir + '/vg'
testdir_nvg = testdir + '/nvg'

train_vg_data = os.listdir(traindir_vg)
train_nvg_data = os.listdir(traindir_nvg)

test_vg_data = os.listdir(testdir_vg)
test_nvg_data = os.listdir(testdir_nvg)

dir_patch_tr=traindir+'/patches'
dir_patch_te=testdir+'/patches'


In [30]:
filename = rootdir+'/final_model_svm'
clf = pickle.load(open(filename, 'rb'))

##Data Preparation

In [40]:
test_data = []
test_data.extend(test_vg_data)
test_data.extend(test_nvg_data)
print(len(test_data))


y_1 = [1]*25
y_0 = [0]*42

y_true=[]
y_true.extend(y_1)
y_true.extend(y_0)
print(len(y_true))

dirpaths = []

vg_path_arr = ['/content/drive/MyDrive/Colab Notebooks/29. Identification of Van Gogh paintings/vgdb_2016/vgdb_2016/test/vg']*25

nvg_path_arr = ['/content/drive/MyDrive/Colab Notebooks/29. Identification of Van Gogh paintings/vgdb_2016/vgdb_2016/test/nvg']*42

dirpaths.extend(vg_path_arr)
dirpaths.extend(nvg_path_arr)

67
67


In [41]:
import random
sample_test_indices = random.sample(range(1, 66), 20)

print(len(sample_test_indices))
print(sample_test_indices)

20
[10, 18, 43, 7, 41, 5, 44, 50, 26, 21, 46, 49, 47, 32, 20, 25, 62, 8, 30, 6]


In [42]:
sample_test_data = []
sample_y_true = []
sample_dir_path = []

for idx in sample_test_indices:
  sample_test_data.append(test_data[idx])
  sample_y_true.append(y_true[idx])
  sample_dir_path.append(dirpaths[idx])

##VGG19

In [46]:
vgg19model = VGG19(include_top = False, weights = 'imagenet')

In [7]:
vgg19model.save(rootdir+'/vgg19model_pre_tflite.h5')

In [8]:
fileSize = os.path.getsize(rootdir+'/vgg19model_pre_tflite.h5')
print('File size: ' + str(round(fileSize / (1024 * 1024), 3)) + ' Megabytes')

File size: 76.455 Megabytes


In [69]:
#pred_values = get_predictions(test_data,dirpaths,False)
pred_values = get_predictions(sample_test_data,sample_dir_path,False,no_of_patches=5)

100%|██████████| 20/20 [00:13<00:00,  1.53it/s]


In [70]:
f1_median,f1_mean,f1_far = get_f1(pred_values,sample_y_true)
print("F1 scores with different Fusion methods")
print("="*200)
print("F1 score for test data - Median is ", f1_median)
print("F1 score for test data - Mean is", f1_mean)
print("F1 score for test data - Far is", f1_far)


F1 scores with different Fusion methods
F1 score for test data - Median is  0.875
F1 score for test data - Mean is 0.875
F1 score for test data - Far is 0.9333333333333333


##Post Quantization

In [31]:
vgg19model1 = VGG19(include_top = False, weights = 'imagenet')

In [32]:
converter = tf.lite.TFLiteConverter.from_keras_model(vgg19model1)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
vgg19_tflite_model = converter.convert()
open(rootdir+'/custom_vgg_model.tflite', "wb").write(vgg19_tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpfh8crd4q/assets


INFO:tensorflow:Assets written to: /tmp/tmpfh8crd4q/assets


20161440

In [33]:
fileSize = os.path.getsize(rootdir+'/custom_vgg_model.tflite')
print('File size: ' + str(round(fileSize / (1024 * 1024), 3)) + ' Megabytes')

File size: 19.227 Megabytes


In [34]:
interpreter = tf.lite.Interpreter(model_content=vgg19_tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details= interpreter.get_output_details()
print("Input Shape", input_details[0]['shape'])
print("Input Type", input_details[0]['dtype'])
print("Output Shape", output_details[0]['shape'])
print("Output Type", output_details[0]['dtype'])

Input Shape [1 1 1 3]
Input Type <class 'numpy.float32'>
Output Shape [  1   0   0 512]
Output Type <class 'numpy.float32'>


In [36]:
interpreter.resize_tensor_input(input_details[0]['index'],(1, 224, 224, 3))
interpreter.resize_tensor_input(output_details[0]['index'], (1, 25088))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()



In [44]:
pred_values_tflite = get_predictions(sample_test_data,sample_dir_path,True,no_of_patches=5)

100%|██████████| 20/20 [1:37:01<00:00, 291.09s/it]


Tflite took more than 20 minutes to predict for the 20 patches per image. Since we have 67 images, it would have taken more than 10 hours to complete execution. Hence, reduced the number of patches to 5.

In [45]:
f1_median,f1_mean,f1_far = get_f1(pred_values_tflite,sample_y_true)
print("F1 scores with different Fusion methods")
print("="*200)
print("F1 score for test data - Median is ", f1_median)
print("F1 score for test data - Mean is", f1_mean)
print("F1 score for test data - Far is", f1_far)

F1 scores with different Fusion methods
F1 score for test data - Median is  0.875
F1 score for test data - Mean is 0.875
F1 score for test data - Far is 0.875


##Utility Functions

In [37]:
no_of_patches =20

'''This function is used to generate patches for a particular image. 
The number of patches and patch_size are passed as function parameters.
The function generates the number of patches as the given parameter'''
def get_patches_for_img(img_path, patch_size, no_of_patches):
    
    patches = []

    #read the image at the given path
    img = cv2.imread(img_path, 1)    
    image_height = img.shape[0]
    image_width = img.shape[1]

    #Subtract patch_size from image's height and width to avoid out of bounds error
    range_x = image_height - patch_size
    range_y = image_width - patch_size

    #Generate patches for each image. The number of patches are passed as parameter.
    for i in range(no_of_patches):
        
        #Generate patch from random area of the image
        x = np.random.randint(low = 0, high = range_x)
        y = np.random.randint(low = 0, high = range_y)

        #The patch is calculated by adding the patch_size to both x and y co-ordinates
        patch = img[x : x+patch_size, y : y+patch_size, :]
        patches.append(patch)

    return patches

'''This function creates a feature array for a patch. 
It uses the VGG 19 to pre-process and predict the feature array for the patch. '''
def get_patch_feature_vgg19_tflite(patch):  
    
    patch_pre_list = []

    #for patch in patches:

    patch_input = np.expand_dims(patch, axis = 0)             #add an extra dimension for batch
    patch_preprocessed_input = preprocess_input(patch_input)    

    #  patch_pre_list.append(patch_preprocessed_input)  
    interpreter.set_tensor(input_details[0]['index'], patch_preprocessed_input)
    interpreter.invoke()
    
    tflite_model_predictions = interpreter.get_tensor(output_details[0]['index'])    
    
    p_feature = tflite_model_predictions.reshape(25088)

    return p_feature

'''This function creates a feature array for a patch. 
It uses the VGG 19 to pre-process and predict the feature array for the patch. '''
def get_patch_feature_vgg19(patch):
    
    patch_input = np.expand_dims(patch, axis = 0)             #add an extra dimension for batch
    patch_preprocessed_input = preprocess_input(patch_input)               

    p_feature = vgg19model.predict(patch_preprocessed_input) 
    p_feature = p_feature.reshape(25088)

    return p_feature


In [38]:
#Fusion methods 

''',
Find the farthest point from the prediction probabilities for each image
'''
def agg_pred_far(pred):

  arr_pos = []
  arr_neg = []
  
  for predItem in pred:
    if(predItem >= 0):
      arr_pos.append(predItem)
    else:
      arr_neg.append(predItem)

  
  max_pos = np.max(arr_pos) if(len(arr_pos) > 0) else 0  
  
  max_neg = np.abs(np.min(arr_neg)) if(len(arr_neg) > 0) else 0

  cl = 1 if(max_pos > max_neg) else 0
  
  return cl

'''
Find the mean from the prediction probabilities for each image
'''
def agg_pred_mean(pred):
  arr_pos = []
  arr_neg = []

  for predItem in pred:
    if(predItem >= 0):
      arr_pos.append(predItem)
    else:
      arr_neg.append(predItem)


  #arr_pos = pred[pred >= 0]
  avg_pos = np.mean(arr_pos) if(len(arr_pos) > 0) else 0
  
  #arr_neg = pred[pred <= 0]
  avg_neg = np.abs(np.mean(arr_neg)) if(len(arr_neg) > 0) else 0

  cl = 1 if(avg_pos > avg_neg) else 0
  
  return cl

'''
Find the median from the prediction probabilities for each image
'''
def agg_pred_median(pred):

  arr_pos = []
  arr_neg = []

  for predItem in pred:
    if(predItem >= 0):
      arr_pos.append(predItem)
    else:
      arr_neg.append(predItem)
  #arr_pos = pred[pred >= 0]
  avg_pos = np.median(arr_pos) if(len(arr_pos) > 0) else 0
  
  #arr_neg = pred[pred <= 0]
  avg_neg = np.abs(np.median(arr_neg)) if(len(arr_neg) > 0) else 0

  cl = 1 if(avg_pos > avg_neg) else 0
  
  return cl


##Prediction Functions

In [39]:
''' This function can be used to obtain the predictions when the filenames, directory paths and actual values are passed.
The number of patches by default are 20 but can be changed.
The predictions are obtained by reading the image from the given directory path, generate random patches ,
use VGG19 to classify the patches and then use SVM to generate probabilities.These probabilities are returned as the 
model's predictions. 
 '''

from tqdm import tqdm

patch_size = 224                            #patch_size for vgg19 model
def get_predictions(filename_list,dirnameArr, isTflite,no_of_patches=20):
  
  predictions=[]
  
  idx = 0
  for file_name in tqdm(filename_list): 
    patch_pred = []
                         
    
    img_path = dirnameArr[idx] + '/' + file_name
    patches = get_patches_for_img(img_path, patch_size, no_of_patches)
    
    idx = idx + 1
    for patch in patches:                                

        if(isTflite):
            patch_feature = get_patch_feature_vgg19_tflite(patch)
        else:
            patch_feature = get_patch_feature_vgg19(patch)
        
        pred_proba = clf.decision_function([patch_feature])      
        
        patch_pred.append(pred_proba)  
        

    predictions.append(patch_pred)                                  
    

  return  predictions 

''' This function can be used to obtain the f1 scores when the filenames, directory paths and actual values are passed.
The number of patches by default are 20 but can be changed.
The predictions are obtained by reading the image from the given directory path, generate random patches ,
use VGG19 to classify the patches and then use SVM to generate probabilities. 
The F1-scores are computed over three fusion methods - Median, Mean and Far. The results from these three
methods are returned. 
 '''

def get_metric_f1_score(filename_list,dirnameArr,y_true,no_of_patches=20):
  
  #get the predictions
  predictions = get_predictions(filename_list,dirnameArr,no_of_patches)   

  y_pred_mean= []
  y_pred_median= []
  y_pred_far=[]
  f1_score_median=[]
  f1_score_mean=[]
  f1_score_far = []
  
  #Compute the F1-scores

  for item in predictions:
    y_pred_median.append(agg_pred_median(item))
    y_pred_mean.append(agg_pred_mean(item))
    y_pred_far.append(agg_pred_far(item))
    
  f1_score_median = f1_score(y_true, y_pred_median)
  f1_score_mean = f1_score(y_true, y_pred_mean)
  f1_score_far = f1_score(y_true, y_pred_far)
  
  return f1_score_median,f1_score_mean,f1_score_far


''' This function can be used to obtain the f1 scores when predictions and the actual values are passed.
The F1-scores are computed over three fusion methods - Median, Mean and Far. The results from these three
methods are returned. 
 '''
def get_f1(predictions,y_true):
  
  y_pred_mean= []
  y_pred_median= []
  y_pred_far=[]
  f1_score_median=[]
  f1_score_mean=[]
  f1_score_far = []
  

  for item in predictions:
    y_pred_median.append(agg_pred_median(item))
    y_pred_mean.append(agg_pred_mean(item))
    y_pred_far.append(agg_pred_far(item))
    
  f1_score_median = f1_score(y_true, y_pred_median)
  f1_score_mean = f1_score(y_true, y_pred_mean)
  f1_score_far = f1_score(y_true, y_pred_far)
  
  return f1_score_median,f1_score_mean,f1_score_far

Observations:

1. The file size before quantization was around 76 MB. After quantization, the file size shrunk to 19MB i.e it became 4 times smaller than the original file.

2. Performance wise, both the models i.e original model and the quantized model yielded good results. Owing to the extremely long execution time of the quantized model, the number of patches had to be reduced to 5 from 20. Even with fewer patches, both models yielded good results. Hence, we can say that the quantization did not drastically reduce the effectiveness of the model.

3. Tflite took more than 20 minutes to predict for the 20 patches per image. Since we have 67 images, it would have taken more than 10 hours to complete execution. Hence, reduced the number of patches to 5.